In [1]:
import PyPDF2
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure

from langdetect import detect
import pandas as pd

In [2]:
def rus(text, alphabet=set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')):
    return not alphabet.isdisjoint(text.lower())

In [3]:
def eng(text, alphabet=set('abcdefghijklmnopqrstuvwxyz')):
    return not alphabet.isdisjoint(text.lower())

In [4]:
def find_extract(text: str, text_to_find: str, bias_=0, without_new_str=False, until=None):

    idx = text.find(text_to_find) + len(text_to_find) + bias_

    list_ = []

    if until is None:
        while not(eng(text[idx])):
            if text[idx] != '\n':
                list_.append(text[idx])
            else:
                if without_new_str:
                    break
            idx += 1
    else:
        while not(text[idx:].find(until) == 0):
            list_.append(text[idx])
            idx += 1

    return(''.join(list_[:-1]))

In [5]:
def parse_marks(text):

    idx = 0
    list_ = []

    for i in range(len(text)):
        if rus(text[i]) or text[i] == ' ':
            list_.append(text[i])

    list_ = ''.join(list_).split(' ')

    return [x for x in list_ if x in ('удовлетворительно', 'хорошо', 'отлично', 'зачтено')]

In [6]:
def parse_diploma_supplement(pdf_path, marks_count=47):

    pdf_file = open(pdf_path, 'rb')
    text = extract_text(pdf_file, password='pass', page_numbers=None, maxpages=0, caching=True, codec='utf-8', laparams=None)

    general_info = pd.DataFrame([], columns=['family_name', 'date_of_birth', 'access_requirements', 'admitted_to', 'graduated_from', 'level_of_education', 'name_of_qualification', 'main_field_of_study', 'academic_programme_title', 'professional_trajectory',
                               'official_length_of_programme', 'mode_of_study', 'languages_of_instruction_examinations', 'graduation_project_theme', 'scientific_supervisor', 'graduation_project_mark'])
    marks = pd.DataFrame([], columns=['family_name'] + [f'subject_{i}' for i in range(1, marks_count + 1)])        

    general_info.loc[len(general_info)] = [
        find_extract(text, 'Family name(s), given name(s) '),
        find_extract(text, 'Дата рождения (день/месяц/год)  '),
        find_extract(text, 'Access requirements '),
        find_extract(text, 'Admitted to '),
        find_extract(text, 'Graduated from '),
        find_extract(text, 'Level of education '),
        find_extract(text, 'Name of qualification '),
        find_extract(text, 'Main field of study '),
        find_extract(text, 'Academic programme title '),
        find_extract(text, 'Профессиональная траектория (профессиональные траектории): '),
        find_extract(text, 'Нормативный срок освоения ', without_new_str=True),
        find_extract(text, 'Форма обучения  ', without_new_str=True),
        find_extract(text, 'Language(s) of instruction / \nexaminations \n\n', without_new_str=True, until='/'),
        find_extract(text, 'Completion and defense \nof graduation project \n\n', until='\nНаучный руководитель').replace('\n', ''),
        find_extract(text, 'Научный руководитель: '),
        find_extract(text[text.find('ECTS \ngrade \n\nGrade') + len('ECTS \ngrade \n\nGrade'):], 'ECTS \ngrade \n\nGrade ')[4:]
    ]
    
    marks_begin_idx = text[text.find('Конец перечня / End of the list'):].find('\x0c') + text.find('Конец перечня / End of the list')
    marks_begin_idx = min(text[marks_begin_idx:].find('удовлетворительно'), text[marks_begin_idx:].find('хорошо'), text[marks_begin_idx:].find('отлично'), text[marks_begin_idx:].find('зачтено')) + marks_begin_idx
    marks_end_idx = text[marks_begin_idx:].find('Конец перечня') + marks_begin_idx
    marks.loc[len(marks)] = [general_info.loc[len(marks)]['family_name']] + parse_marks(text[marks_begin_idx:marks_end_idx])

    return general_info, marks

In [7]:
pdf_path = '../data/files/РыбаковСергейВячеславович_Supl-1.pdf'
marks_count = 47

In [8]:
general_info, marks = parse_diploma_supplement(pdf_path, marks_count)

In [9]:
general_info

,family_name,date_of_birth,access_requirements,admitted_to,graduated_from,level_of_education,name_of_qualification,main_field_of_study,academic_programme_title,professional_trajectory,official_length_of_programme,mode_of_study,languages_of_instruction_examinations,graduation_project_theme,scientific_supervisor,graduation_project_mark
0,Рыбаков Сергей Вячеславович,05.07.2000,"Аттестат о среднем общем образовании, выданный...",Федеральное государственное бюджетное образова...,Федеральном государственном бюджетном образова...,бакалавриат,Бакалавр,01.03.02 Прикладная математика и информатика,"Прикладная математика, фундаментальная информа...",Математическое и программное обеспечение вычис...,4 года,очная,"English, Russian",Разработка масштабируемой архитектуры игрового...,кандидат технических наук И.С.Блеканов,отлично


In [10]:
marks

,family_name,subject_1,subject_2,subject_3,subject_4,subject_5,subject_6,subject_7,subject_8,subject_9,...,subject_38,subject_39,subject_40,subject_41,subject_42,subject_43,subject_44,subject_45,subject_46,subject_47
0,Рыбаков Сергей Вячеславович,отлично,удовлетворительно,хорошо,зачтено,зачтено,зачтено,удовлетворительно,зачтено,хорошо,...,зачтено,хорошо,отлично,хорошо,зачтено,отлично,зачтено,отлично,отлично,зачтено
